In [1]:
from music21 import *
from hmmlearn import hmm
from sklearn.externals import joblib

In [2]:
S = corpus.getComposer('bach')
all_parsed = []
num = 1
for p in S:
    
    all_parsed.append(converter.parse(p))
    
    print('finished', num, 'of', len(S))
    num += 1

finished 1 of 426
finished 2 of 426
finished 3 of 426
finished 4 of 426
finished 5 of 426
finished 6 of 426
finished 7 of 426
finished 8 of 426
finished 9 of 426
finished 10 of 426
finished 11 of 426
finished 12 of 426
finished 13 of 426
finished 14 of 426
finished 15 of 426
finished 16 of 426
finished 17 of 426
finished 18 of 426
finished 19 of 426
finished 20 of 426
finished 21 of 426
finished 22 of 426
finished 23 of 426
finished 24 of 426
finished 25 of 426
finished 26 of 426
finished 27 of 426
finished 28 of 426
finished 29 of 426
finished 30 of 426
finished 31 of 426
finished 32 of 426
finished 33 of 426
finished 34 of 426
finished 35 of 426
finished 36 of 426
finished 37 of 426
finished 38 of 426
finished 39 of 426
finished 40 of 426
finished 41 of 426
finished 42 of 426
finished 43 of 426
finished 44 of 426
finished 45 of 426
finished 46 of 426
finished 47 of 426
finished 48 of 426
finished 49 of 426
finished 50 of 426
finished 51 of 426
finished 52 of 426
finished 53 of 426
fi

finished 417 of 426
finished 418 of 426
finished 419 of 426
finished 420 of 426
finished 421 of 426
finished 422 of 426
finished 423 of 426
finished 424 of 426
finished 425 of 426
finished 426 of 426


In [3]:
joblib.dump(S, '/home/daniel/Desktop/RhythmHMM/bach_parsed.pkl', compress=9)

['/home/daniel/Desktop/RhythmHMM/bach_parsed.pkl']

In [4]:
corpus_offsets = []
corpus_durations = []
for s in all_parsed:
    all_offsets = []
    all_durations = []
    measure = 0
    measure_offset = 0
#     print(type(s))
    if type(s) is stream.Score:
        if len(s.parts) > 1:
            for S in s.parts:
                for thisNote in S.flat.getElementsByClass(["Note", "Rest"]):
                #     print(thisNote, thisNote.offset)
                    if thisNote.offset not in all_offsets:
                        all_offsets.append(thisNote.offset)
                        all_durations.append(thisNote.quarterLength)
                corpus_offsets.append(all_offsets)
                corpus_durations.append(all_durations)
    if type(s) is stream.Opus:
        for SS in s:
            for S in SS.parts:
                for thisNote in S.flat.getElementsByClass(["Note", "Rest"]):
                #     print(thisNote, thisNote.offset)
                    if thisNote.offset not in all_offsets:
                        all_offsets.append(thisNote.offset)
                        all_durations.append(thisNote.quarterLength)
                corpus_offsets.append(all_offsets)
                corpus_durations.append(all_durations)

In [5]:
corpus_matrix = []
for all_offsets, all_durations in zip(corpus_offsets, corpus_durations):
    total_measures = int(all_offsets[-1])#/4 if measures wanted
    offset_matrix = []
    measure = 0
    beat = 1/4
    max_beats = 4.0
    for meas in range(total_measures):
        beat_measure = []
        sustain = 0
        for x in range(4):
            if meas+x/4 in all_offsets:
                beat_measure.append(1)
                sustain += all_durations[all_offsets.index(meas+x/4)]-1/4
#                 print('sustain increase', sustain)
            elif sustain > 0:
                beat_measure.append(0)
                sustain = sustain-1/4
#                 print('sustain decrease', sustain)
            else:
                beat_measure.append(-1)
        offset_matrix.append(beat_measure)
#         print('finished', meas, 'of', total_measures)
    corpus_matrix.append(offset_matrix)

In [6]:
rhythm_dict = {}
rhythm_number = 0
for x in corpus_matrix:
    for y in x:
        if y not in rhythm_dict.values():
            rhythm_dict[rhythm_number] = y
            rhythm_number += 1

In [ ]:
# len(rhythm_dict)
corpus_matrix[0]

In [7]:
for a, b in rhythm_dict.items():
    print(a, b)

0 [1, 0, 0, 0]
1 [1, 0, 1, 0]
2 [1, 1, 1, 1]
3 [1, 0, 1, 1]
4 [-1, -1, -1, -1]
5 [1, 1, 1, 0]
6 [-1, -1, 1, 0]
7 [1, 0, 0, 1]
8 [-1, 1, 1, 0]


In [ ]:
all_rhythms = []
song_lengths = []
for x in corpus_matrix:
    song_lengths.append(len(x))
    for xx in x:
        for a, b in rhythm_dict.items():
            if xx == b:
                all_rhythms.append([a])

In [ ]:
Z = hmm.MultinomialHMM(n_components=3, verbose=True,n_iter=1000,tol=.01)
Z.n_features = len(rhythm_dict)

In [ ]:
r_fit = Z.fit(all_rhythms, song_lengths)

In [ ]:
emit = r_fit.emissionprob_

In [ ]:
import threading
from multiprocessing import Process
from queue import Queue
import time, os

import numpy as np
# from hmmlearn import hmm
# import alfabeto_data.dissertation_images as di
# from alfabeto_data.harmonic_function_data import ordered_numeral_list
# from alfabeto_sources.all_sources import *
# from alfabeto_sources import *
# from alfabeto_code.AlfabetoConverter import transposed_pc_chords_noMMD
# from Continuo.ContinuoConverter import figure_intervals_pc
import csv
import copy

from sklearn.externals import joblib
import pickle
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
from scipy.spatial.distance import euclidean


def hmm_pickle_generator(hmm_data, hmm_lengths, number_features, corpus_name, MC, K, RANGE, start_path):
    shuffle_path = start_path+'hmm_mc/%s/%s/%s' % (corpus_name, K, MC)
    if not os.path.exists(shuffle_path):
        os.makedirs(shuffle_path)

    if os.path.exists(start_path+'hmm_mc/%s/%s/%sMC.pkl' % (corpus_name, K, MC)):
        print('already MC there')
        MC_data = joblib.load(start_path+'hmm_mc/%s/%s/%sMC.pkl' % (corpus_name, K, MC))
        X = MC_data['X']
        X_train = MC_data['X_train']
        X_test = MC_data['X_test']
    else:
        all_x = []
        starting = 0
        for x in hmm_lengths:
            all_x.append([a for a in hmm_data[starting:starting+x]])
#         print('all_x', all_x)
        X = [(a, b) for a, b in zip(all_x, hmm_lengths)]
        
        np.random.shuffle(X)
        train_percent = int(len(X)/5)
        X_train = X[train_percent:]
#         print('x_train', X_train)
        X_test = X[:train_percent]
        decode_data = []
        pickle_data = {'X': X, 'X_train': X_train, 'X_test': X_test}
        joblib.dump(pickle_data, shuffle_path + 'MC.pkl', compress=9)
        
    Z = hmm.MultinomialHMM(n_components=K, verbose=False, n_iter=500, tol=.01)
    pickle_path = start_path+'hmm_mc/%s/%s/%s' % (corpus_name, K, MC)
    #pickle_path = 'pickles/hmm/%s/%s/%s' % (corpus_name, mode_name, K)
    if not os.path.exists(pickle_path):
        os.makedirs(pickle_path)
    for worker in RANGE:
        path = pickle_path+'/%s.pkl' % worker
    if not os.path.exists(path):
#         print('starting!', K, worker)
        s_time = time.time()
        Y = copy.deepcopy(Z)
        Y.n_features = number_features
        X_train_multinomial = []
        X_train_lengths = []
        for tr in X_train:
#             print(tr)
            for a in tr[0]:
                X_train_multinomial.append(a)
#             print('x_train_m', X_train_multinomial)
            X_train_lengths.append(tr[1])
#             print('x_train_l', X_train_lengths)

        X_test_multinomial = []
        X_test_lengths = []
        for tr in X_test:
#             print(tr)
            for a in tr[0]:
                X_test_multinomial.append(a)
#             print('x_test_m', X_test_multinomial)
            X_test_lengths.append(tr[1])
#             print('x_test_l', X_test_lengths)
            
#         print('x_multi', X_train_multinomial)
#         print('x_len', X_train_lengths)
#         print('XTM', X_train_multinomial)
#         print(len(X_train_multinomial))
#         print(sum(X_train_lengths))
#         print('XTL', X_train_lengths)
        a = Y.fit(X_train_multinomial, X_train_lengths)
        c = a.decode(X_test_multinomial, lengths=X_test_lengths, algorithm='viterbi')
        dict_data = {'train': X_train, 'test': X_test, 'fit': a, 'decode': c, 'total_chords': len(X[2])}
        e_time = time.time() - s_time
        print('K=%s number %s done in %s seconds' % (K, worker, e_time))
        joblib.dump(dict_data, path, compress=9)
    else:
        print(K, worker, 'already exists... :)')

In [ ]:
hmm_pickle_generator(all_rhythms, song_lengths, len(rhythm_dict), 'mozart_beat', 5, 3, [2, 3, 4, 5], '/home/daniel/Desktop/RhythmHMM/')

In [ ]:
thing = 0
for cluster in range(2, 16):
        for mc in range(5):
            for mc_range in range(50):
                thing += 1
                print(cluster, mc, mc_range)
                hmm_pickle_generator(all_rhythms, song_lengths, len(rhythm_dict), 
                                     'mozart', mc, cluster, [mc_range], '/home/daniel/Desktop/RhythmHMM/')

In [ ]:
from multiprocessing import Pool
s_path = '/home/daniel/Desktop/RhythmHMM/hmm_mc/'
composer = 'palestrina_beat'
parallel_dict = {}
thing_number = 0
for cluster in range(2, 9):
    for mc in range(5):
        for mc_range in range(50):
            if not os.path.exists(s_path+composer+'/%s/%s/%s.pkl' %(cluster, mc, mc_range)):
#                 print(s_path+composer+'/%s/%s/%s' %(cluster, mc, mc_range))
                parallel_dict[thing_number] = {'cluster': cluster, 'mc': mc, 'mc_range': [mc_range]}
                thing_number += 1
print('number left:', len(parallel_dict))
def f(x):
    hmm_pickle_generator(all_rhythms, song_lengths, len(rhythm_dict), 'palestrina_beat', 
                         parallel_dict[x]['mc'], 
                         parallel_dict[x]['cluster'], 
                         parallel_dict[x]['mc_range'], 
                         '/home/daniel/Desktop/RhythmHMM/')


with Pool(8) as p:
    print(p.map(f, [x for x in parallel_dict.keys()]))

In [ ]:
import numpy as np
import time
from sklearn.externals import joblib
from scipy.spatial.distance import cdist
from alfabeto_data.dissertation_images import k_means_simple

def silhouette_hmm(silhouette_number):
    all_silhouette = {}
    thenumber = 0
    for corpus in ['palestrina_beat']:
        for mcc in range(5):
            all_silhouette[thenumber] = {'corpus': corpus, 'mc': mcc}
            thenumber += 1
    corpus_name = all_silhouette[silhouette_number]['corpus']
    
    mc = all_silhouette[silhouette_number]['mc']
    K_range = 8
    flipped_data = {}
    for K in range(2, K_range+1):
        all_data = []
        all_labels = []
        l_number = joblib.load('/home/daniel/Desktop/RhythmHMM/hmm_mc/%s/%s/%s/0.pkl' % (corpus_name, K, mc))['test']
        l_numeral = rhythm_dict
#         l_numeral = joblib.load('/home/daniel/Desktop/RhythmHMM/hmm_mc/%s/%s/%s/0.pkl' % (corpus_name, K, mc))['train'][2]
#         print('l num', l_number)
        for l in l_number:
#             print(l[0])
            for x in l[0]:
                all_labels.append(str(x))
#             for ll in l:
#                 all_labels.append(l_numeral[a])
#                 print(ll)
#                     if ll == a:
#                         all_labels.append(b)
#     #             if l[0] < len(l_numeral):
#     #                 all_labels.append(l_numeral[l[0]])
#                     else:
#                         all_labels.append('o')
#                         print('one didn\'t fit...')
        for x in range(50):
            temp_data = []
#             print(x)
            c = joblib.load('/home/daniel/Desktop/RhythmHMM/hmm_mc/%s/%s/%s/%s.pkl' % (corpus_name, K, mc, x))['decode']
#             print('decoded')
            for xx in c[1]:
                temp_data.append(xx)
            all_data.append(temp_data)
        numpied = np.swapaxes(np.array(all_data), 0, 1)
    #     print(numpied.shape)
        st = time.time()
        DIST = cdist(numpied, numpied, 'hamming')
#         DIST = (cdist(numpied, numpied, 'hamming')*len(l_number))**2 #Quinn/White
        st2 = time.time()
#         print('LEN DIST', len(DIST))
#         print('LEN LABS', len(all_labels))
        flipped_data[K] = k_means_simple(DIST, K, all_labels)
        
    #     print_labels = ['$'+x+'$' for x in all_labels]
    #     di.k_means_data(DIST, K, print_labels, '/home/daniel/Desktop/hmmkmeans/%s_%s.pdf' % (corpus, K))
    #     print('kmeans took', time.time()-st2)
        print(K, '-->', flipped_data[K])
    return {(corpus_name, mc): flipped_data}

In [ ]:
# silhouette_hmm(1)
# with Pool(8) as p:
#     print(p.map(silhouette_hmm, [x for x in range(5)]))
for x in range(5):
    print(silhouette_hmm(x))